In [1]:
%run optimizers/Optimizer.ipynb

class AlternatingPoseShapeEstimation(Optimizer):
    
    def __init__(self, mean, pcaBasis, pcaVariance, numIterations=3):
        Optimizer.__init__(self, mean, pcaBasis, pcaVariance)
        self.numIterations = numIterations
        self.ai, self.ri, self.ti, self.si = Optimizer.getDefaults(self.numComponents)
    
    def open(self):
        return
    
    def close(self):
        return
    
    def optimize(self, targetFeatures):
        for _ in range(self.numIterations):
            self.ri, self.ti, self.si = self._initializePoseParameters(targetFeatures, self.ai)
            self.ai = self._initializeShapeParameters(targetFeatures, self.ri, self.ti, self.si)
        return self.ai, self.ri, self.ti, self.si
    
    def update(self, a, r, t, s):
        self.ai = a
        self.ri = r
        self.ti = t
        self.si = s
    
    def _initializePoseParameters(self, targetFeatures, a):
        
        # calculate [u1, v2, w3, ..., un, vn, wn]
        model_features = self.mean + np.matmul(self.pcaBasis, a)

        # A component buffer
        A = np.zeros((2*self.numFeatures, 8))

        # initiaze A
        for i in range(self.numFeatures):
            A[2*i,:]   = [model_features[3*i], model_features[3*i+1], model_features[3*i+2], 1, 0, 0, 0, 0]
            A[2*i+1,:] = [0, 0, 0, 0, model_features[3*i], model_features[3*i+1], model_features[3*i+2], 1]

        # solve the linear equation
        k = lsq_linear(A, targetFeatures, lsq_solver='exact')['x'].reshape((8,1))

        r1 = np.array([k[0][0], k[1][0], k[2][0]])
        r2 = np.array([k[4][0], k[5][0], k[6][0]])

        s = (np.linalg.norm(r1) + np.linalg.norm(r2)) / 2.
        t = np.array([k[3][0]/s, k[7][0]/s]).reshape((2,1))

        U,S,V = np.linalg.svd(np.array([r1, r2, np.cross(r1, r2)]))
        r = np.matmul(U,V) # TODO, should .T??
        if np.linalg.det(r) == -1:
            U,S,V = np.linalg.svd(np.array([r1, r2, -1*np.cross(r1, r2)]))
            r = np.matmul(U,V.T) # TODO, should .T??
        
        return r, t, s
    
    def _initializeShapeParameters(self, targetFeatures, r, t, s):

        # C and h component data buffers
        C = np.zeros((2*self.numFeatures, self.numComponents))
        h = np.zeros((2*self.numFeatures, 1))

        # initialize C
        for i in range(self.numFeatures):
            C[2*i]   = s * (r[0,0] * self.pcaBasis[3*i,:] + r[0,1] * self.pcaBasis[3*i+1,:] + r[0,2] * self.pcaBasis[3*i+2,:])
            C[2*i+1] = s * (r[1,0] * self.pcaBasis[3*i,:] + r[1,1] * self.pcaBasis[3*i+1,:] + r[1,2] * self.pcaBasis[3*i+2,:])
            h[2*i]   = targetFeatures[2*i] - s * (np.matmul(r[0], self.mean[3*i:3*i+3]) + t[0,0])
            h[2*i+1] = targetFeatures[2*i+1] - s * (np.matmul(r[1], self.mean[3*i:3*i+3]) + t[1,0])

        # return the result
        std = np.sqrt(self.pcaVariance.reshape(self.numComponents,))   # TODO: pass std not variance to function
        bounds = (-3*std, 3*std)
        return lsq_linear(C, h.flatten(), bounds=bounds)['x'].reshape((self.numComponents,1)) # TODO, constraint

ERROR:root:File `'optimizers/Optimizer.ipynb.py'` not found.


NameError: name 'Optimizer' is not defined